In [1]:
import  numpy as np
import  torch
import  pandas as pd
import DataSet_2

In [4]:
FaceData=DataSet_2.TorchFacialFeaturesDataset();

In [22]:
generator=torch.Generator().manual_seed(69)
train_size = int(0.85 * len(FaceData))
test_size = len(FaceData) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(FaceData, [train_size, test_size],generator=generator)

4280